In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from concurrent.futures import ThreadPoolExecutor
from time import sleep
import pandas as pd
import os

def init_driver():
    """Khởi tạo trình duyệt Chrome cho mỗi luồng."""
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.maximize_window()
    return driver

def scroll_page(driver, scroll_count=10, pause=1.5, offset=25):
    """Cuộn trang để tải thêm nội dung."""
    total_height = driver.execute_script("return document.body.scrollHeight")
    scroll_step = (total_height - offset) / scroll_count
    for i in range(1, scroll_count + 1):
        driver.execute_script(f"window.scrollTo(0, {i * scroll_step});")
        sleep(pause)

def load_bookmark(bookmark_file="bookmark.xlsx"):
    """Tải danh sách các liên kết đã cào từ file Excel."""
    if os.path.exists(bookmark_file):
        df = pd.read_excel(bookmark_file)
        return set(df["Link"].tolist())
    return set()

def save_bookmark(collected_links, bookmark_file="bookmark.xlsx"):
    """Lưu danh sách các liên kết đã cào vào file Excel."""
    df = pd.DataFrame({"Link": list(collected_links)})
    df.to_excel(bookmark_file, index=False)

def append_to_excel(new_data, file_path):
    """Nối thêm dữ liệu mới vào file Excel mà không ghi đè."""
    if os.path.exists(file_path):
        # Đọc dữ liệu cũ nếu file đã tồn tại
        old_data = pd.read_excel(file_path)
        # Nối dữ liệu cũ và mới
        combined_data = pd.concat([old_data, new_data], ignore_index=True)
    else:
        # Nếu file chưa tồn tại, chỉ lưu dữ liệu mới
        combined_data = new_data
    
    # Ghi dữ liệu kết hợp vào file Excel
    combined_data.to_excel(file_path, index=False)


def collect_titles_and_links(driver, limit=100, bookmark_file="bookmark.xlsx"):
    """Thu thập tiêu đề và liên kết bài viết."""
    collected_data = []
    collected_links = load_bookmark(bookmark_file)
    print(f"Đã cào {len(collected_links)} bài. Cào thêm đến {limit} bài.")

    while len(collected_links) < limit:
        articles = driver.find_elements(By.CSS_SELECTOR, "article.item-news")
        if not articles:
            scroll_page(driver, scroll_count=10)
            continue
        for article in articles:
            try:
                title_element = article.find_element(By.CSS_SELECTOR, "h3.title-news a")
                title = title_element.text
                link = title_element.get_attribute("href")
                if link not in collected_links:
                    collected_links.add(link)
                    author = article.find_element(By.CSS_SELECTOR, "p.meta-news a.name-author").text
                    author_link = article.find_element(By.CSS_SELECTOR, "p.meta-news a.name-author").get_attribute("href")
                    category = article.find_element(By.CSS_SELECTOR, "p.meta-news a.cat").text
                    comments_count = article.find_element(By.CSS_SELECTOR, "p.meta-news a.count_cmt span.font_icon").text
                    collected_data.append({
                        "Title": title,
                        "Link": link,
                        "Author": author,
                        "Author Link": author_link,
                        "Category": category,
                        "Comments Count": comments_count
                    })
                    print(f"{len(collected_links)}/{limit}")  # In tiến trình cào
                    save_bookmark(collected_links, bookmark_file)  # Cập nhật bookmark
                    if len(collected_links) >= limit:
                        break
            except NoSuchElementException:
                continue
        scroll_page(driver, scroll_count=5)
    return collected_data


def extract_detail_info(driver, link):
    """Thu thập thông tin chi tiết từ một bài viết."""
    driver.get(link)
    sleep(2)
    try:
        date = driver.find_element(By.CSS_SELECTOR, "span.date").text
        title_detail = driver.find_element(By.CSS_SELECTOR, "h1.title-detail").text
        description = driver.find_element(By.CSS_SELECTOR, "p.description").text
        content = "\n".join([p.text for p in driver.find_elements(By.CSS_SELECTOR, "article.fck_detail p")])

        # Cuộn trang để tải toàn bộ nội dung
        scroll_page(driver, scroll_count=10, pause=1.5, offset=50)

        # Thu thập bình luận
        comments_data = []
        total_comments_count = 0

        while True:  # Nhấn vào "Xem thêm ý kiến" nếu tồn tại
            try:
                view_more_button = driver.find_element(By.CSS_SELECTOR, "div.view_more_coment a#show_more_coment")
                driver.execute_script("arguments[0].click();", view_more_button)
                sleep(2)
                driver.execute_script("window.scrollBy(0, 500);")
                sleep(1)
            except NoSuchElementException:
                break

        comments = driver.find_elements(By.CSS_SELECTOR, "div.comment_item")
        for comment in comments:
            try:
                nickname = comment.find_element(By.CSS_SELECTOR, "span.txt-name a.nickname").text
                nickname_href = comment.find_element(By.CSS_SELECTOR, "span.txt-name a.nickname").get_attribute("href")
                comment_content = comment.find_element(By.CSS_SELECTOR, "p.full_content").text.strip()
                likes = comment.find_element(By.CSS_SELECTOR, "div.reactions-total a.number").text
                comment_time = comment.find_element(By.CSS_SELECTOR, "span.time-com").text
                reply_nicknames, reply_links = [], []
                try:
                    reply_button = comment.find_element(By.CSS_SELECTOR, "a.view_all_reply")
                    driver.execute_script("arguments[0].click();", reply_button)
                    sleep(1)
                    replies = comment.find_elements(By.CSS_SELECTOR, "div.sub_comment div.comment_item")
                    for reply in replies:
                        reply_nicknames.append(reply.find_element(By.CSS_SELECTOR, "span.txt-name a.nickname").text)
                        reply_links.append(reply.find_element(By.CSS_SELECTOR, "span.txt-name a.nickname").get_attribute("href"))
                except NoSuchElementException:
                    pass
                comments_data.append({
                    "Nickname": nickname,
                    "Nickname Link": nickname_href,
                    "Comment": comment_content,
                    "Likes": likes,
                    "Comment Time": comment_time,
                    "Reply Nicknames": reply_nicknames,
                    "Reply Links": reply_links
                })
                total_comments_count += 1
            except NoSuchElementException:
                continue

        return {
            "Date": date,
            "Detailed Title": title_detail,
            "Description": description,
            "Content": content,
            "Comments Count": total_comments_count,
            "Comments": comments_data
        }
    except NoSuchElementException:
        return {}


def process_article(article, idx, total):
    """Xử lý từng bài viết trong một luồng và in tiến trình."""
    print(f"Processing {idx + 1}/{total}: {article['Link']}")
    driver = init_driver()
    try:
        details = extract_detail_info(driver, article["Link"])
        article.update(details)
    finally:
        driver.quit()
    return article


def main():
    # Bước 1: Khởi tạo trình duyệt và thu thập tiêu đề và liên kết
    driver = init_driver()
    driver.get("https://vnexpress.net/goc-nhin")
    articles = collect_titles_and_links(driver, limit=1000, bookmark_file="bookmark.xlsx")
    driver.quit()

    # Bước 2: Xử lý thông tin chi tiết bằng đa luồng
    total_articles = len(articles)
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(lambda args: process_article(*args), 
                                    [(article, idx, total_articles) for idx, article in enumerate(articles)]))

    # Lưu dữ liệu vào Excel
    df = pd.DataFrame(results)
    print(df)
    append_to_excel(df, "D:/du lieu o cu/HUTECH Courses/Social Networking Course/SocialNetworkingProject/Project của Đạt/vnexpress_articles_4.xlsx")

if __name__ == "__main__":
    main()


Đã cào 600 bài. Cào thêm đến 1000 bài.
601/1000
602/1000
603/1000
604/1000
605/1000
606/1000
607/1000
608/1000
609/1000
610/1000
611/1000
612/1000
613/1000
614/1000
615/1000
616/1000
617/1000
618/1000
619/1000
620/1000
621/1000
622/1000
623/1000
624/1000
625/1000
626/1000
627/1000
628/1000
629/1000
630/1000
631/1000
632/1000
633/1000
634/1000
635/1000
636/1000
637/1000
638/1000
639/1000
640/1000
641/1000
642/1000
643/1000
644/1000
645/1000
646/1000
647/1000
648/1000
649/1000
650/1000
651/1000
652/1000
653/1000
654/1000
655/1000
656/1000
657/1000
658/1000
659/1000
660/1000
661/1000
662/1000
663/1000
664/1000
665/1000
666/1000
667/1000
668/1000
669/1000
670/1000
671/1000
672/1000
673/1000
674/1000
675/1000
676/1000
677/1000
678/1000
679/1000
680/1000
681/1000
682/1000
683/1000
684/1000
685/1000
686/1000
687/1000
688/1000
689/1000
690/1000
691/1000
692/1000
693/1000
694/1000
695/1000
696/1000
697/1000
698/1000
699/1000
700/1000
701/1000
702/1000
703/1000
704/1000
705/1000
706/1000
707/100